In [201]:
# Google drive mout
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [202]:
import pandas as pd
import re
import math
import requests
from bs4 import BeautifulSoup

In [203]:
# 장르 리스트 만들기
for page in range(1,2):
  url = "https://www.vgchartz.com/games/games.php?page={}&results=200&genre=Action&order=Sales&ownership=Both&direction=DESC&showtotalsales=1&shownasales=1&showpalsales=1&showjapansales=1&showothersales=1&showpublisher=1&showdeveloper=0&showreleasedate=1&showlastupdate=0&showvgchartzscore=1&showcriticscore=0&showuserscore=0&showshipped=1".format(page)
  res = requests.get(url)
  res.raise_for_status()
  soup = BeautifulSoup(res.text, "lxml")

  genre = soup.find('select', attrs={"name":"genre"})

genre_list = []
for i in genre.find_all('option')[1:]:
  genre_list.append(i.string)

In [204]:
genre_list

['Action',
 'Action-Adventure',
 'Adventure',
 'Board Game',
 'Education',
 'Fighting',
 'Misc',
 'MMO',
 'Music',
 'Party',
 'Platform',
 'Puzzle',
 'Racing',
 'Role-Playing',
 'Sandbox',
 'Shooter',
 'Simulation',
 'Sports',
 'Strategy',
 'Visual Novel']

In [205]:
# ['Action', 'Action-Adventure', 'Adventure', 'Board Game', 'Education', 'Fighting', 'Misc', 'MMO', 'Music', 'Party', 'Platform']
# ['Puzzle', 'Racing', 'Role-Playing', 'Sandbox', 'Shooter', 'Simulation', 'Sports', 'Strategy', 'Visual Novel']
# genre_list = ['Puzzle', 'Racing', 'Role-Playing', 'Sandbox', 'Shooter', 'Simulation', 'Sports', 'Strategy', 'Visual Novel']

In [206]:
# 빈컬럼 만들기
Name = []
Platform = []
Year = []
Genre = []
Publisher = []
Total_Shipped = []
Total_Sales = []
NA_Sales = []
PAL_Sales = []
JP_Sales = []
Other_Sales = []

In [207]:
# 장르별 페이지 수 계산
for gen in genre_list:
  url = "https://www.vgchartz.com/games/games.php?page=1&results=200&genre={}&order=Sales&ownership=Both&direction=DESC&showtotalsales=1&shownasales=1&showpalsales=1&showjapansales=1&showothersales=1&showpublisher=1&showdeveloper=0&showreleasedate=1&showlastupdate=0&showvgchartzscore=1&showcriticscore=0&showuserscore=0&showshipped=1".format(gen)
  res = requests.get(url)
  res.raise_for_status()
  soup = BeautifulSoup(res.text, "lxml")

  chart = soup.select_one('div#generalBody > table')
  result_str = chart.find_all('tr')[0].th.string
  pages = math.floor(int(re.sub(r'[^0-9]', '', result_str[result_str.find('(')+1 : result_str.find(')')])) / 200) + 2
  print(gen, ' 📖 total pages : ', pages)

# 스트래핑
  for page in range(1, pages):
    url = "https://www.vgchartz.com/games/games.php?page={}&results=200&genre={}&order=Sales&ownership=Both&direction=DESC&showtotalsales=1&shownasales=1&showpalsales=1&showjapansales=1&showothersales=1&showpublisher=1&showdeveloper=0&showreleasedate=1&showlastupdate=0&showvgchartzscore=1&showcriticscore=0&showuserscore=0&showshipped=1".format(page, gen)
    res = requests.get(url)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, "lxml")

    chart = soup.select_one('div#generalBody > table')

    for row in chart.find_all('tr')[3:]:
      col = row.select('td')
      
      Name.append(col[2].a.string)
      Platform.append(col[3].find("img")['alt'])
      Publisher.append(col[4].string)
      Total_Shipped.append(col[6].string)
      Total_Sales.append(col[7].string)
      NA_Sales.append(col[8].string)
      PAL_Sales.append(col[9].string)
      JP_Sales.append(col[10].string)
      Other_Sales.append(col[11].string)
      Year.append(col[12].string)
      Genre.append(gen)

    print(type(chart), gen, '🚀 page number : ', page)


Action  📖 total pages :  43
<class 'bs4.element.Tag'> Action 🚀 page number :  1
<class 'bs4.element.Tag'> Action 🚀 page number :  2
<class 'bs4.element.Tag'> Action 🚀 page number :  3
<class 'bs4.element.Tag'> Action 🚀 page number :  4
<class 'bs4.element.Tag'> Action 🚀 page number :  5
<class 'bs4.element.Tag'> Action 🚀 page number :  6
<class 'bs4.element.Tag'> Action 🚀 page number :  7
<class 'bs4.element.Tag'> Action 🚀 page number :  8
<class 'bs4.element.Tag'> Action 🚀 page number :  9
<class 'bs4.element.Tag'> Action 🚀 page number :  10
<class 'bs4.element.Tag'> Action 🚀 page number :  11
<class 'bs4.element.Tag'> Action 🚀 page number :  12
<class 'bs4.element.Tag'> Action 🚀 page number :  13
<class 'bs4.element.Tag'> Action 🚀 page number :  14
<class 'bs4.element.Tag'> Action 🚀 page number :  15
<class 'bs4.element.Tag'> Action 🚀 page number :  16
<class 'bs4.element.Tag'> Action 🚀 page number :  17
<class 'bs4.element.Tag'> Action 🚀 page number :  18
<class 'bs4.element.Tag'> A

In [208]:
columns = {
  'Name' : Name,
  'Platform' : Platform,
  'Year' : Year,
  'Genre' : Genre,
  'Publisher' : Publisher,
  'Total_Shipped' : Total_Shipped,
  'Total_Sales' : Total_Sales,
  'NA_Sales' : NA_Sales,
  'PAL_Sales' : PAL_Sales,
  'JP_Sales' : JP_Sales,
  'Other_Sales' : Other_Sales
  }

In [210]:
df = pd.DataFrame(columns) # 데이터프레임 저장
path = '/content/drive/Shareddrives/coding/aib/AIB_section1_project/data/vgchartz.csv'
df.to_csv(path, sep=',', encoding = 'utf-8')